<H2 style="text-align: center">Convolutional Neural Networks and Transfer Learning</H2>

## Convolutional Neural Networks (CNN)
In this tutorial, we are going to use PyTorch, the cutting-edge deep learning framework to complete our task.

### STL-10 Dataset

Details on STL-10 dataset can be found here: https://cs.stanford.edu/~acoates/stl10

In [ ]:
import torch
import torchvision
## Create dataloader, in PyTorch, we feed the trainer data with use of dataloader
## We create dataloader with dataset from torchvision, 
## and we dont have to download it seperately, all automatically done

# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 256 # We use a small batch size here for training
batch_size_test = 1024 #
# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#image datasets
train_dataset = torchvision.datasets.STL10('dataset/', 
                                           split='train', 
                                           download=True,
                                           transform=image_transform)
test_dataset = torchvision.datasets.STL10('dataset/', 
                                          split='test', 
                                          download=True,
                                          transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True)

###Example Image
Since in the data loader, we already have transformed images, we need to apply inverse transformation on the image to see the original image.

In [ ]:
# import library
import matplotlib.pyplot as plt
#inverse normalization
inv_normalize = torchvision.transforms.Normalize(mean=[-1.0, -1.0, -1.0], std=[1/0.5, 1/0.5, 1/0.5])
# We can check the dataloader
_, (example_datas, labels) = next(enumerate(test_loader))
print(example_datas.shape)
sample = example_datas[0]
# show the data
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(sample.permute(1, 2, 0))
ax2.imshow(inv_normalize(sample).permute(1, 2, 0))
print("Label: " + str(labels[0]))

### CNN Model
You have to define trainable layers and put them inside a model. Have a look on the documentation of [PyTorch](https://pytorch.org/docs/stable/index.html) and read more about different layers and functionalities of PyTorch there. Here we are going to implement various versions of AlexNet model and use it for classification.

In [ ]:
## Now we can start to build our CNN model
## We first import the pytorch nn module and optimizer
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## Then define the model class
class AlexNet1(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet1, self).__init__()
        # input channel 3, output channel 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        # relu non-linearity
        self.relu1 = nn.ReLU()
        # max pooling
        self.max_pool2d1 = nn.MaxPool2d(kernel_size=3, stride=2)
        # input channel 64, output channel 192
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.max_pool2d2 = nn.MaxPool2d(kernel_size=3, stride=2)
        # input channel 192, output channel 384
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        # input channel 384, output channel 256
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        # input channel 256, output channel 256
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        self.max_pool2d5 = nn.MaxPool2d(kernel_size=3, stride=2)
        # adaptive pooling
        self.adapt_pool = nn.AdaptiveAvgPool2d(output_size=(6, 6))
        #dropout layer
        self.dropout1 = nn.Dropout()
        # linear layer
        self.linear1 = nn.Linear(in_features=9216, out_features=4096, bias=True)
        self.relu6 = nn.ReLU()
        self.dropout2 = nn.Dropout()
        self.linear2 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU()
        self.linear3 = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.max_pool2d1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max_pool2d2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.max_pool2d5(x)
        x = self.adapt_pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout1(x)
        x = self.linear1(x)
        x = self.relu6(x)
        x = self.dropout2(x)
        x = self.linear2(x)
        x = self.relu7(x)
        x = self.linear3(x)
        return x

class AlexNet2(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(6, 6))
            )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(in_features=9216, out_features=4096, bias=True),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
            )

    def forward(self, x):
        x = self.features(x)
        # B x C x H x W -> B x C*H*W
        x = x.reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

class AlexNet3(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet3, self).__init__()
        from torchvision import models
        alexnet = models.alexnet(pretrained=False)
        self.features = alexnet.features
        self.avgpool = alexnet.avgpool
        self.classifier = alexnet.classifier
        self.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # B x C x H x W -> B x C*H*W
        x = x.reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

class AlexNet4(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet4, self).__init__()
        from torchvision import models
        alexnet = models.alexnet(pretrained=True)
        self.features = alexnet.features
        self.avgpool = alexnet.avgpool
        self.classifier = alexnet.classifier
        self.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # B x C x H x W -> B x C*H*W
        x = x.reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

### Model and Optimizer Initialization

In [ ]:
## create model and optimizer
learning_rate = 0.0001
weight_decay = 0.0005
# define the model 
model = AlexNet2(10)
# device: cuda or cpu
device = "cuda"
# map to device
model = model.to(device)
# make the parameters trainable
for param in model.parameters():
    param.requires_grad = True
# define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

### Meter
Meter for keeping losses and accuracies

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### Train and Test Functions

In [ ]:
from tqdm import tqdm_notebook as tqdm
##define train function
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    # meter
    loss = AverageMeter()
    # switch to train mode
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)  # data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss_this = F.cross_entropy(output, target)
        loss_this.backward()
        optimizer.step()
        loss.update(loss_this.item(), target.shape[0])
    print('Train: Average loss: {:.4f}\n'.format(loss.avg))
        
##define test function
def test(model, device, test_loader):
    # meters
    loss = AverageMeter()
    acc = AverageMeter()
    # switch to test mode
    correct = 0
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # data, target = data.cuda(), target.cuda()
            output = model(data)
            loss_this = F.cross_entropy(output, target) # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct_this = pred.eq(target.view_as(pred)).sum().item()
            correct += correct_this
            acc_this = correct_this/target.shape[0]*100.0
            acc.update(acc_this, target.shape[0])
            loss.update(loss_this.item(), target.shape[0])
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss.avg, correct, len(test_loader.dataset), acc.avg))

### Training Loop
Training loop containing alternating train and test phase

In [ ]:
num_epoch = 20
for epoch in range(1, num_epoch + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

### Summary
Show the summary of the model

In [ ]:
print(model)

In [ ]:
from torchsummary import summary
summary(model, (3, 96, 96))

##Transfer Learning

### TU-Berlin Dataset
Details on TU-Berlin dataset can be found here: http://cybertron.cg.tu-berlin.de/eitz/projects/classifysketch/

In [ ]:
import os
if not os.path.exists('sketches_png.zip'):
    !wget http://cybertron.cg.tu-berlin.de/eitz/projects/classifysketch/sketches_png.zip
    !unzip -q sketches_png.zip
    !rm sketches_png.zip
    !mv png tu_berlin

### Split into Train and Test Set

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
with open('tu_berlin/filelist.txt', 'r') as fp:
    files = fp.read().splitlines()
classes_str = [file.split('/')[0] for file in files]
classes = np.unique(classes_str, return_inverse=True)[1]
train_files, test_files, train_classes, test_classes = train_test_split(files, classes, train_size=0.3, test_size=0.1, stratify=classes)

### Data Generator

In [ ]:
from PIL import Image
import torch.utils.data as data
class TUBerlin(data.Dataset):
    def __init__(self, root, files, classes, transform=None):
        self.root = root
        self.files = files
        self.classes = classes
        self.transform = transform

    def __getitem__(self, item):
        image = Image.open(os.path.join(self.root, self.files[item])).convert(mode='RGB')
        class_ = self.classes[item]
        if self.transform is not None:
            image = self.transform(image)
        return image, class_

    def __len__(self):
        return len(self.files)

###Intialization of the Data Generator

In [ ]:
import torch
import torchvision
# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 256 # We use a small batch size here for training
batch_size_test = 1024 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.Resize((224, 224)),
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#image datasets
train_dataset = TUBerlin('tu_berlin/', train_files, train_classes, 
                         transform=image_transform)
test_dataset = TUBerlin('tu_berlin/', test_files, test_classes, 
                        transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True, num_workers=4)

###Example Image
Since in the data loader, we already have transformed images, we need to apply inverse transformation on the image to see the original image.

In [ ]:
# import library
import matplotlib.pyplot as plt
#inverse normalization
inv_normalize = torchvision.transforms.Normalize(mean=[-1.0, -1.0, -1.0], std=[1/0.5, 1/0.5, 1/0.5])
# We can check the dataloader
_, (example_datas, labels) = next(enumerate(train_loader))
print(example_datas.shape)
sample = example_datas[0]
# show the data
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(sample.permute(1, 2, 0))
ax2.imshow(inv_normalize(sample).permute(1, 2, 0))
print("Label: " + str(labels[0]))

### Model and Optimizer Initialization

In [ ]:
## create model and optimizer
learning_rate = 0.0001
weight_decay = 0.0005
# define the model 
model = AlexNet4(250)
# device: cuda or cpu
device = "cuda"
# map to device
model = model.to(device)
# make the parameters trainable
for param in model.parameters():
    param.requires_grad = True
# define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

### Training Loop
Training loop containing alternating train and test phase

In [ ]:
num_epoch = 20
for epoch in range(1, num_epoch + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))